In [20]:
def get_billable_miles(actual_miles: int, config: dict) -> int:
    """
    Calculate billable miles using either 'pattern' or 'tiers' mode.
    """
    mode = config["mode"]

    if mode == "tiers":
        for rng, values in config["tiers"].items():
            start, end = map(int, rng.split("-"))
            if start <= actual_miles <= end:
                return values["billable_miles"]
        return actual_miles  # fallback if not in any tier

    elif mode == "pattern":
        rules = config["pattern"]

        # Free range (0–5 miles no charge)
        if actual_miles <= 5:
            return 0

        start = rules["start"]
        step = rules["first_step"]
        cap = start + step - 1

        while cap < rules["max_miles"]:
            if actual_miles <= cap:
                return cap
            step += rules["step_growth"]   # widen the interval
            start = cap + 1
            cap = start + step - 1

        return rules["max_miles"]

    else:
        raise ValueError(f"Unknown mileage mode: {mode}")

In [21]:
rules = {
  "mode": "pattern",
  "pattern": {
    "start": 6,
    "first_step": 2,
    "step_growth": 1,
    "max_miles": 1000,
    "free_range": {
      "min": 0,
      "max": 5,
      "billable_miles": 0
    }
  }
}

In [28]:
get_billable_miles(16,rules)

19

In [12]:
dynamic_modifiers = {
  "vehicle_location": {
    "Interstate": {
      "label": "Interstate",
      "lanes": {
        "Left Shoulder": { "label": "Left Shoulder", "upcharge": 0.15 },
        "Right Shoulder": { "label": "Right Shoulder", "upcharge": 0.0 },
        "Median": { "label": "Median", "upcharge": 0.20 }
      }
    },
    "Highway": {
      "label": "Highway",
      "lanes": {
        "Left Shoulder": { "label": "Left Shoulder", "upcharge": 0.15 },
        "Right Shoulder": { "label": "Right Shoulder", "upcharge": 0.0 },
        "Median": { "label": "Median", "upcharge": 0.20 }
      }
    },
    "Two Lane": {
      "label": "Two Lane",
      "lanes": {
        "Side of road": { "label": "Side of road", "upcharge": 0.0 },
        "Middle of road": { "label": "Middle of road", "upcharge": 0.20 }
      }
    }
  },

  "weather": {
    "rain": { "label": "Rain", "upcharge": 0.10 },
    "thunderstorm_lightening": { "label": "ThunderStorm/Lightening", "upcharge": 0.15 },
    "tornado_hurricane_hail": { "label": "Tornado/Hurricane/Hail", "upcharge": 0.25 },
    "clear": { "label": "Clear", "upcharge": 0.0 },
    "low_visibility": { "label": "Low Visibility", "upcharge": 0.10 },
    "flood": { "label": "Flood", "upcharge": 0.20 },
    "ice_snow": { "label": "Ice/Snow", "upcharge": 0.25 },
    "extreme_heat": { "label": "Extreme Heat", "upcharge": 0.15 }
  },

  "truck_utilization": {
    "80%": { "threshold": 80, "upcharge": 0.05 },
    "85%": { "threshold": 85, "upcharge": 0.07 },
    "90%": { "threshold": 90, "upcharge": 0.10 },
    "95%": { "threshold": 95, "upcharge": 0.15 },
    "100%": { "threshold": 100, "upcharge": 0.20 }
  },

  "time_of_day": {
    "07:00-08:00": { "start": "07:00", "end": "08:00", "upcharge": 0.10 },
    "08:00-09:00": { "start": "08:00", "end": "09:00", "upcharge": 0.15 },
    "09:00-10:00": { "start": "09:00", "end": "10:00", "upcharge": 0.12 },
    "15:00-16:00": { "start": "15:00", "end": "16:00", "upcharge": 0.10 },
    "16:00-17:00": { "start": "16:00", "end": "17:00", "upcharge": 0.15 },
    "17:00-18:00": { "start": "17:00", "end": "18:00", "upcharge": 0.0 }
  },

  "bucket_mileage_pricing": {
    "mode": "pattern",
    "pattern": {
      "start": 6,
      "first_step": 2,
      "step_growth": 1,
      "max_miles": 1000,
      "free_range": { "min": 0, "max": 5, "billable_miles": 0 }
    }
  }
}


In [11]:
def get_billable_miles(actual_miles: int, dynamic_modifiers: dict) -> int:
    """
    Calculate billable miles using either 'pattern' or 'tiers' mode.
    """
    mode = dynamic_modifiers["bucket_mileage_pricing"]["mode"]

    if mode == "tiers":
        for rng, values in config["tiers"].items():
            start, end = map(int, rng.split("-"))
            if start <= actual_miles <= end:
                return values["billable_miles"]
        return actual_miles  # fallback if not in any tier

    elif mode == "pattern":
        rules = dynamic_modifiers["bucket_mileage_pricing"]["pattern"]
        # Free range (0–5 miles no charge)
        if actual_miles <= 5:
            return 0

        start = rules["start"]
        step = rules["first_step"]
        cap = start + step - 1

        while cap < rules["max_miles"]:
            if actual_miles <= cap:
                return cap
            step += rules["step_growth"]   # widen the interval
            start = cap + 1
            cap = start + step - 1

        return rules["max_miles"]

    else:
        raise ValueError(f"Unknown mileage mode: {mode}")

In [19]:
get_billable_miles(71,dynamic_modifiers)

82

In [1]:
import os
import pandas as pd
import json

# go up one level from current working dir (same pattern as your pricing script)
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
DATA_DIR = os.path.join(BASE_DIR, "data")

CSV_FILE = os.path.join(DATA_DIR, "cars.csv")
JSON_FILE = os.path.join(DATA_DIR, "cars.json")

print("Looking for CSV at:", CSV_FILE)

df = pd.read_csv(CSV_FILE, sep="\t|,", engine="python")  # supports tab or comma CSVs

cars = {}

for _, row in df.iterrows():
    make = str(row["make"]).strip()
    model = str(row["model"]).strip()
    car_type = str(row["car_type"]).strip()
    upcharge = float(row["upcharge_percentage"])

    # Initialize make if not present
    if make not in cars:
        cars[make] = {
            "label": make,
            "models": {}
        }

    # Add model under make
    cars[make]["models"][model] = {
        "label": model,
        "car_type": car_type,
        "upcharge_percentage": upcharge
    }

# Save JSON
os.makedirs(DATA_DIR, exist_ok=True)
with open(JSON_FILE, "w") as f:
    json.dump(cars, f, indent=2)

print(f"✅ cars.json created successfully at {JSON_FILE}")


ModuleNotFoundError: No module named 'pandas'